# House Price Prediction

Let's look at supervised learning in a classic ML course example: predicting house prices.

Our goal is going to build a prediction model to predict the sale price of a house based on easily observed features. The data are from the Seattle, WA area for a few months in 2014.



We start by importing packages we will be using. See [our first example](https://colab.research.google.com/github/chansen776/MBA-ML-Course-Materials/blob/main/Code/BiasVarianceExample1.ipynb).


In [ ]:
# Install extra libraries
!pip install formulaic
from formulaic import model_matrix

# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import make_scorer
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree


# Seed we will use for random number generators
rng = 713


# Load and examine the data

We need to load the data. We will do this directly from a github repository for the course.

In [ ]:
file = "https://raw.githubusercontent.com/chansen776/MBA-ML-Course-Materials/main/Data/WAHousePrice.xlsx"
data = pd.read_excel(file)
data.shape  # See size of dataset

Let's take a look at what's in the data

In [ ]:
data.columns

In [ ]:
data.describe()

The maximum price 2.659000e+07 = 26,590,000 seems very high. We'll look at this again in a second. There are also some very low prices and properties with 0 bedrooms and bathrooms. Nothing else in the summary statistics stands out as particularly noteworthy.

Let's quickly examine the 0 bedroom and 0 bathroom observations.

In [ ]:
# Let's find the observations with 0 bedrooms or 0 bathrooms and see what they look like
data[(data['bedrooms'] == 0) | (data['bathrooms'] == 0)]

I'm pretty confident that houses with 3000+ square feet of living space should have bathrooms and bedrooms. We could try to *impute* what seem to be the missing values of bathrooms and bedrooms for these two observations. Because it's only two observations, I'm just going to delete them instead.

In [ ]:
# Drop the flagged observations
data = data.drop(data[(data['bedrooms'] == 0) | (data['bathrooms'] == 0)].index)

In [ ]:
data.dtypes

Some of the variables `street`, `city`, `statezip`, and `country` are not numeric, so don't show up in the summary table. We'll look at these variables after examining the outcome in a bit more detail.

**Outcome variable**

Let's start by seeing what the outcome variable looks like.

In [ ]:
# Smoothed histogram of the outcome variable, price
sns.displot(data=data, x='price', kind='kde')
plt.title('Smoothed histogram of Price')
plt.show()

In [ ]:
plt.scatter(data['sqft_living'], data['price'])
plt.xlabel('sqft_living')
plt.ylabel('price')
plt.title('Scatter plot of Price vs Square Feet')
plt.show()

There are two observations that are probably worth another look. Also recall that there was at least one very low price from the summary statistics. Let's look at both.

In [ ]:
# Let's find the observations with very low prices and see what they look like
data[data['price'] < 50000]

It seems a bit out of the realm of possibility to have a $7800 dollar property in the Seattle area in 2014, though the property seems not particularly desirable. I'm going to assume that the price here is incorrect.

In [ ]:
# Let's find the observations with very high prices and see what they look like
data[data['price'] > 5000000]


I can believe that a 10000 square foot property sold for $7M.

I have a hard time believing a 2 bathroom, 3 bedroom, 1180 square foot property sold for ~\$27M. This seems like a mistake. I'm also having a hard time believing that a 2.5 bathroom, 3 bedroom, 2190 square foot property sold for ~$13M.

I am going to treat both of these observations as "mistakes" and drop them from my data. I am going to recognize that if these are real observations, there is a small chance I will see future properties that I make GIGANTIC mistakes on.

We can/should also run the whole thing with the dropped observations included to gauge robustness.

In [ ]:
# Drop the flagged observations
data = data.drop(data[data['price'] < 50000].index)
data = data.drop(data[data['price'] > 10000000].index)

Our price variable is highly skewed. It is very commeon to see data-scientists build models a right-skewed variable that only takes on positive values by taking a `log` transformation first. I.e. instead by building a model for `price`, we might instead build a model for `log(price)`.

**Remember that our goal is to predict `price` though!** Make sure you validate on the basis of what you care about.

In [ ]:
# Create new column with the log of price
data['log_price'] = np.log(data['price'])

# Smoothed histogram of log price
sns.displot(data=data, x='log_price', kind='kde')
plt.title('Smoothed histogram of log(Price)')
plt.show()

In [ ]:
plt.scatter(data['sqft_living'], data['log_price'])
plt.xlabel('sqft_living')
plt.ylabel('price')
plt.title('Scatter plot of log(Price) vs Square Feet')
plt.show()

**Categorical Variables**

The four variables of type "object" are not numeric but coded as text. Let's look at what they are. In the following code blocks, we are just going to tabulate each of our categorical variables by looking at the `value_counts` property of our data.

In [ ]:
data['street'].value_counts()

Most street addresses have only one property. Without better geographic knowledge, it's going to be hard to use this variable.

**Question:** If you really cared, how could you use the street address to get something potentially more useful? (Think about, e.g., google street view, google maps and routing, ...)

In [ ]:
data['city'].value_counts()

City might be useful, but it is very unbalanced.

**Questions:**

1. Do we think the actual name of the city matters? Why might the variable `city` contain useful information?

2. If we want to predict something outside of the Seattle area, is the variable `city` useful? Could we make it useful with some more effort?

In [ ]:
data['statezip'].value_counts()

Zip code (`statezip`) seems like it might have more *interesting* variation than `city`.

**Questions:**

1. Do we think the actual numeric value of a property's zip code matters? Why might the variable `statezip` contain useful information?

2. If we want to predict something outside of the Seattle area, is the variable `statezip` useful? Could we make it useful with some more effort?

In [ ]:
data['country'].value_counts()

Can't use the variable `country` - It doesn't *vary* 🙂.

**Aside: Categorical Variables.**

The typical way to use (unordered) categorical variables is to encode them as dummy variables/binary variables/use one-hot-encoding --  all of which are jargon for making a new set of variables that are 0 or 1 with 1 indicating the observation belongs to a category and 0 indicates it does not.

For example, if we had a variable `color` with two categories "red" and "blue", we just create two new variables `red` and `blue` where variable `red` = 1 for all observations that are red (and 0 otherwise) and `blue` is defined similarly. (Note: For standard linear models, you typically exclude one of the dummy variables. In our toy example, we don't need both the variables `red` and `blue` because if `red` = 1 we know `blue` = 0 and viceversa. That is, they have the same information.)

We'll look at doing this in our price example when we consider including `city` and `statezip` in our model.

Before doing anything else, let's drop the variables we definitely won't use.

In [ ]:
data = data.drop(columns=['date','street','country'])

Finally, we also see that we have many other variables that are effectively capturing qualitative information or a mix. E.g. `bathrooms`, `bedrooms`, `floors`, `waterfront`, `view`, and `condition` might be thought of as categorical.

Let's look at these variables more carefully.

In [ ]:
print(data['bathrooms'].value_counts())
print('\n')
print(data['bedrooms'].value_counts())
print('\n')
print(data['floors'].value_counts())
print('\n')
print(data['waterfront'].value_counts())
print('\n')
print(data['view'].value_counts())
print('\n')
print(data['condition'].value_counts())

Importantly, these variables all have an order to them. `bathrooms`, `bedrooms`, and `floors` also (arguably) have cardinal value. `view` and `condition` are clearly qualitative, but do have ordinal meaning. `waterfront` is already a dummy variable.

We could treat these by including them as dummy variables, but it also makes sense to use them as is (which is what we are going to do). Many ML procedures will happily and appropriately deal with ordered categorical features and numeric features with few values.

**Mixed variables**

The variables `sqft_basement` and `yr_renovated` have a 0 category indicating "no basement" or "never renovated" respectively.

We can add variables indicating the qualitative information. For many learners, adding this variable is also unnecessary.

In [ ]:
# Add a variable for being renovated
data['renovated_flag'] = np.where(data['yr_renovated'] == 0, 0, 1)

# Add a variable for having a basement
data['basement_flag'] = np.where(data['sqft_basement'] == 0, 0, 1)

Do the the square footage variables `sqft_living`, `sqft_above` and `sqft_basement` capture different information?

In [ ]:
# Let's calculate the correlation of sqft_living with sqft_above+sqft_basement
print('Correlation of sqft_living with (sqft_above+sqft_basement):',
      data['sqft_living'].corr(data['sqft_above']+data['sqft_basement']))

# Let's calculate the correlation with each element instead
print('Correlation of sqft_living with sqft_above:',
      data['sqft_living'].corr(data['sqft_above']))
print('Correlation of sqft_living with sqft_basement:',
      data['sqft_living'].corr(data['sqft_basement']))


Cannot use `sqft_living` and both `sqft_basement` and `sqft_above` in the same linear model estimated by least squares (or least absolute values).

**Question:** How do we choose which to use?

# Model evaluation

We are going to use 5-fold CV to evaluate our models. We want to keep the cross-validation folds the same across all of the models we build, so we're going to predefine the splits.

In [ ]:
cvsplit = KFold(n_splits=5, shuffle=True, random_state=rng)

We also need to choose how to evaluate the models. For illustration, we're going to consider performance based on both MAE and MSE. We are going to estimate models using both `price` and `log_price` for illustration.

When we build a model for `log_price`, we don't directly build a prediction rule for `price`. We are going to use the simplest approach to turning our `log_price` prediction into a `price` prediction by exponentiating. I.e. if $\widehat{\texttt{log_price}}$ is our prediction for `log_price`, we obtain a prediction for `price` as $\exp\{\widehat{\texttt{log_price}}\}$.

We're going to define functions to produce MSE and MAE from `log_price` predictions.

In [ ]:
def expmse(y_true, y_pred):

  y_true = y_true.to_numpy()
  y_pred[y_pred < -20] = -20
  y_pred[y_pred > 20] = 20 # Prevent overflow issues in really bad models because we are going to exponentiate
  negmse = -np.mean((np.exp(y_true) - np.exp(y_pred))**2)

  return negmse

# Create a scorer object using the expmse function
expmse_score = make_scorer(expmse)

def expmae(y_true, y_pred):

  y_true = y_true.to_numpy()
  y_pred[y_pred < -20] = -20
  y_pred[y_pred > 20] = 20 # Prevent overflow issues in really bad models because we are going to exponentiate
  negmae = -np.mean(np.abs(np.exp(y_true) - np.exp(y_pred)))

  return negmae

# Create a scorer object using the expmae function
expmae_score = make_scorer(expmae)


# Baseline start

As simple benchmark, let's look at sample means and medians.

We're going to define functions for using the mean and median as prediction rules that we can use with `sklearn`'s built in cross-validation functions.

In [ ]:
# Define function to use mean as estimator and make prediction
class MeanEstimator(BaseEstimator, RegressorMixin):
    def fit(self, X, y=None):
        # Compute the mean of y during fitting
        self.mean_ = np.mean(y)
        return self

    def predict(self, X):
        # Return the mean for all predictions
        return np.full(len(X), self.mean_)

# Define function to use median as estimator and make prediction
class MedianEstimator(BaseEstimator, RegressorMixin):
    def fit(self, X, y=None):
        # Compute the mean of y during fitting
        self.median_ = np.median(y)
        return self

    def predict(self, X):
        # Return the mean for all predictions
        return np.full(len(X), self.median_)


Let's look at how well the sample mean and sample median of `price` do for predicting `price` by 5-fold cross-validation.

In [ ]:
# Cross validation RMSE and MAE using sample mean of price as prediction rule
# The "scoring" argument tells cross-validation what out-of-sample prediction
# metrics to compute.
levmean_mse = cross_validate(MeanEstimator(), data['price'], data['price'],
                             scoring=('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('Level Mean: CV RMSE: {m1:=.2f}; CV MAE: {m2:=.2f}'
  .format(m1=np.sqrt(-levmean_mse['test_neg_mean_squared_error'].mean()),
          m2=-levmean_mse['test_neg_mean_absolute_error'].mean()))

# Cross validation RMSE and MAE using sample median of price as prediction rule
# The "scoring" argument tells cross-validation what out-of-sample prediction
# metrics to compute.
levmedian_mse = cross_validate(MedianEstimator(), data['price'], data['price'],
                               scoring=('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('Level Median: CV RMSE: {m1:=.2f}; CV MAE: {m2:=.2f}'
  .format(m1=np.sqrt(-levmedian_mse['test_neg_mean_squared_error'].mean()),
          m2=-levmedian_mse['test_neg_mean_absolute_error'].mean()))

# We are going to use cross-validated MSE from the mean as our benchmark for MSE
# We are going to use cross-validated MAE from the median as our benchmark for MAE
benchMSE = -levmean_mse['test_neg_mean_squared_error'].mean()
benchMAE = -levmedian_mse['test_neg_mean_absolute_error'].mean()

In [ ]:
# Let's create a table to keep track of results
model_results = pd.DataFrame()
model_results['Model'] = ['Mean', 'Median']
model_results['RMSE'] = [np.sqrt(-levmean_mse['test_neg_mean_squared_error'].mean()),
                         np.sqrt(-levmedian_mse['test_neg_mean_squared_error'].mean())]
model_results['R2 - MSE'] = [None,None]
model_results['MAE'] = [-levmean_mse['test_neg_mean_absolute_error'].mean(),
                        -levmedian_mse['test_neg_mean_absolute_error'].mean()]
model_results['R2 - MAE'] = [None,None]
model_results['p'] = [None,None]
model_results['p_use'] = [None,None]
model_results

# Baseline linear models.

## Think about the model

For black box machine learners that discover nonlinearity automatically, thinking about how to model nonlinearity and other transformations to make of our available variables is not a big issue. The ability to put less time into  thinking about these issues is a major plus for ML algorithms. **You should always be thoughtful about variables in a model though.**

For learners like linear regression, thinking about the appropriate way to include variables is potentially very important though.


**Nonlinearity**

We should think about potential *nonlinearities* that we would like to allow our models to capture. For example, do we think that each unit increase in house's square footage at fixed values of other variables should be associated with the same change in predicted (log) price regardless of the square footage? E.g. do we think going from 1000 to 1100 square feet is the same "value" as going from 3000-3100 square feet? Do we think the "value" of going from 2000-2100 square feet is the same if we are talking about a three versus five bedroom property?

A very simple and interpretable way to allow for nonlinearities is to use *polynomials* and *interactions*. E.g. we might include `sqft_living` and `sqft_living`$^2$ to allow for decreasing or increasing "returns" to square footage in our model, so our prediction rule would be

$$\widehat{\texttt{price}} = b_0 + b_1 \texttt{sqft_living} + b_2 \texttt{sqft_living}^2 + ...$$

or

$$\widehat{\texttt{log_price}} = b_0 + b_1 \texttt{sqft_living} + b_2 \texttt{sqft_living}^2 + ...$$

[Note that you cannot interpret $b_1$ and $b_2$ in isolation.]

We might also include the *interaction* `sqft_living`*`bedrooms` to allow the predicted price change for a given change in `sqft_living` to be different for properties with different numbers of bedrooms. In this case, our model would be

$$ \widehat{\texttt{price}} = b_0 + b_1 \texttt{sqft_living} + b_2 \texttt{sqft_living}^2 + b_3\texttt{bedrooms} + b_4 \texttt{sqft_living}*\texttt{bedrooms} + ...$$

or

$$ \widehat{\texttt{log_price}} = b_0 + b_1 \texttt{sqft_living} + b_2 \texttt{sqft_living}^2 + b_3\texttt{bedrooms} + b_4 \texttt{sqft_living}*\texttt{bedrooms} + ...$$

[Note that you cannot interpret $b_1$, $b_2$, $b_3$, and $b_4$ in isolation.]

In our baseline prediction rules we will include squares of all "numeric" variable and include the interaction of `sqft_living` with `bedrooms` and with `bathrooms`

## Baseline models: OLS

In [ ]:
# Here, we're using the "formula" interface we imported from formulaic
# Specify model as formula
# Putting a variable inside "C" will make the variable be treated as categorical
# by turning it into dummy variables
# Putting a variable inside "poly" will a create a polynomial in that variable
# Putting v1:v2 creates the interaction between variables v1 and v2
base = ("price ~ poly(bathrooms, degree=2, raw=True) + poly(bedrooms, degree=2, raw=True)"
             " + poly(sqft_living, degree=2, raw=True) + poly(sqft_lot, degree=2, raw=True) + poly(floors, degree=2, raw=True) "
            "+ waterfront + poly(view, degree=2, raw=True) + poly(condition, degree=2, raw=True) + "
            "poly(yr_built, degree=2, raw=True) + poly(yr_renovated, degree=2, raw=True) + C(renovated_flag) + C(basement_flag)"
            " + sqft_living:(bedrooms+bathrooms)")

# Create variables based on model defined in formula
y, X = model_matrix(base, data)
n, p_base = X.shape

# Cross validation.
baselm = cross_validate(LinearRegression(), X, y,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('Baseline LM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baselm['test_neg_mean_squared_error'].mean()),
          m2=1-(-baselm['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_base))

print('Baseline LM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baselm['test_neg_mean_absolute_error'].mean(),
          m2=1-(-baselm['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_base))

#####################################################################
# Model for log-price
logy = data['log_price']

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation.
baseloglm = cross_validate(LinearRegression(), X, logy,
                       scoring = scoring, cv=cvsplit)
print('Baseline logLM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baseloglm['test_expmse'].mean()),
          m2=1-(-baseloglm['test_expmse'].mean()/benchMSE),
          m3=p_base))

print('Baseline logLM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baseloglm['test_expmae'].mean(),
          m2=1-(-baseloglm['test_expmae'].mean()/benchMAE),
          m3=p_base))


In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Baseline LM',
                                       'RMSE': np.sqrt(-baselm['test_neg_mean_squared_error'].mean()),
                                       'R2 - MSE': 1-(-baselm['test_neg_mean_squared_error'].mean()/benchMSE),
                                       'MAE': -baselm['test_neg_mean_absolute_error'].mean(),
                                       'R2 - MAE': 1-(-baselm['test_neg_mean_absolute_error'].mean()/benchMAE),
                                       'p': p_base,
                                       'p_use': p_base}, ignore_index=True)

model_results = model_results._append({'Model': 'Baseline logLM',
                                       'RMSE': np.sqrt(-baseloglm['test_expmse'].mean()),
                                       'R2 - MSE': 1-(-baseloglm['test_expmse'].mean()/benchMSE),
                                       'MAE': -baseloglm['test_expmae'].mean(),
                                       'R2 - MAE': 1-(-baseloglm['test_expmae'].mean()/benchMAE),
                                       'p': p_base,
                                       'p_use': p_base}, ignore_index=True)

model_results


Not bad. Both models are a sizeable improvement relative to the simple sample mean or median.

Let's look at the `log_price` model refit using the entire dataset in more detail.

In [ ]:
model = sm.OLS(logy, X)
result = model.fit(cov_type='HC3')  # cov_type = 'HC3' computes specification robust standard errors
print(result.summary())

print('\n')
pred_err = result.resid
print("RMSE of model: ", np.sqrt(np.mean(pred_err**2)))
print('\n')

**Questions:**

1. How does the reported $R^2$ relate to the CV $R^2$?
2. What does the reported RMSE mean?
3. How does `sqft_living` relate to the `log_price` prediction? (Ask ChatGPT and see if it gets it right. The last time I tried it was close but not perfect.)

A useful way to think about gauging how variables relate to predictions is just to probe the rule by predicting at some values of interest. This can be especially useful for black-box prediction rules.

Let's specifically look at 2000 vs 2100 square feet and 4 vs 5 bedrooms

In [ ]:
newX = pd.DataFrame(columns = X.columns)
newX = newX._append(X.iloc[125]) # Observation with a 2.5 bathroom, 4 bedroom, 2000 square foot house
newX = pd.DataFrame(np.repeat(newX.values, 4, axis=0))
newX.columns = X.columns
newX.iloc[1,5] = 2100
newX.iloc[1,6] = 2100**2
newX.iloc[1,22] = 4*2100
newX.iloc[1,23] = 2.5*2100
newX.iloc[2,3] = 5
newX.iloc[2,4] = 5**2
newX.iloc[2,22] = 5*2000
newX.iloc[3,3] = 5
newX.iloc[3,4] = 5**2
newX.iloc[3,5] = 2100
newX.iloc[3,6] = 2100**2
newX.iloc[3,22] = 5*2100
newX.iloc[3,23] = 2.5*2100

# Predicted log prices for our four hypothetical houses
yhat = result.predict(newX)
print('Difference in predicted log(price) 2100 vs 2000 sqft (4 beds):', yhat[1]-yhat[0])
print('Difference in predicted log(price) 2100 vs 2000 sqft (5 beds):', yhat[3]-yhat[2])
print('Difference in predicted log(price) 4 vs 5 beds (2000 sqft):', yhat[2]-yhat[0])
print('Difference in predicted log(price) 4 vs 5 beds (2100 sqft):', yhat[3]-yhat[1])
print('\n')
print('Difference in predicted price 2100 vs 2000 sqft (4 beds):', np.exp(yhat[1])-np.exp(yhat[0]))
print('Difference in predicted price 2100 vs 2000 sqft (5 beds):', np.exp(yhat[3])-np.exp(yhat[2]))
print('Difference in predicted price 4 vs 5 beds (2000 sqft):', np.exp(yhat[2])-np.exp(yhat[0]))
print('Difference in predicted price 4 vs 5 beds (2100 sqft):', np.exp(yhat[3])-np.exp(yhat[1]))



# Baseline linear models: LAV regression

We can run least absolute values regression using `sklearn`'s `QuantileRegressor`

In [ ]:
from sklearn.linear_model import QuantileRegressor

model = make_pipeline(StandardScaler(), QuantileRegressor(alpha = 0, solver = 'highs-ds'))
# Lots of learners work best with variables on a common scale. StandardScaler()
# standardizes the data. The pipeline is going to standardize before running
# the quantile regression.

# Cross validation.
baseqr = cross_validate(model, X, y.to_numpy().ravel(),
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('Baseline LAV: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baseqr['test_neg_mean_squared_error'].mean()),
          m2=1-(-baseqr['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_base))

print('Baseline LAV: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baseqr['test_neg_mean_absolute_error'].mean(),
          m2=1-(-baseqr['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_base))

#####################################################################
# Model for log-price
# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation.
baselogqr = cross_validate(model, X, logy,
                       scoring = scoring, cv=cvsplit)
print('Baseline logLAV: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baselogqr['test_expmse'].mean()),
          m2=1-(-baselogqr['test_expmse'].mean()/benchMSE),
          m3=p_base))

print('Baseline logLAV: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baselogqr['test_expmae'].mean(),
          m2=1-(-baselogqr['test_expmae'].mean()/benchMAE),
          m3=p_base))

In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Baseline LAV',
                                       'RMSE': np.sqrt(-baseqr['test_neg_mean_squared_error'].mean()),
                                       'R2 - MSE': 1-(-baseqr['test_neg_mean_squared_error'].mean()/benchMSE),
                                       'MAE': -baseqr['test_neg_mean_absolute_error'].mean(),
                                       'R2 - MAE': 1-(-baseqr['test_neg_mean_absolute_error'].mean()/benchMAE),
                                       'p': p_base,
                                       'p_use': p_base}, ignore_index=True)

model_results = model_results._append({'Model': 'Baseline logLAV',
                                       'RMSE': np.sqrt(-baselogqr['test_expmse'].mean()),
                                       'R2 - MSE': 1-(-baselogqr['test_expmse'].mean()/benchMSE),
                                       'MAE': -baselogqr['test_expmae'].mean(),
                                       'R2 - MAE': 1-(-baselogqr['test_expmae'].mean()/benchMAE),
                                       'p': p_base,
                                       'p_use': p_base}, ignore_index=True)

model_results

# A More Flexible Model

Maybe we haven't captured all the nonlinearity with our quadratic polynomials. Maybe there are interactions between variables besides the couple we thought about (`sqft_living` with `bedrooms` and `bathrooms`).

Let's try a more elaborate model.

In [ ]:
# Specify model as formula
# Putting a variable inside "C" will make the variable be treated as categorical
# by turning it into dummy variables
# Putting a variable inside "poly" will a create a polynomial in that variable
flex = ("price ~ poly(bathrooms, degree=4, raw=True) + poly(bedrooms, degree=4, raw=True)"
             " + poly(sqft_living, degree=6, raw=True) + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag)"
             " + poly(bathrooms, degree=4, raw=True):(poly(bedrooms, degree=4, raw=True)"
             " + poly(sqft_living, degree=6, raw=True) + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(bedrooms, degree=4, raw=True):(poly(sqft_living, degree=6, raw=True)"
             " + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(sqft_living, degree=6, raw=True):(poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(sqft_lot, degree=4, raw=True):(poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(floors, degree=4, raw=True):(waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + waterfront:(C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + C(view):(C(condition) + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + C(condition):(poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(yr_built, degree=4, raw=True):(poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(yr_renovated, degree=4, raw=True):(C(renovated_flag) + C(basement_flag))"
             " + C(renovated_flag):C(basement_flag)")

# Create variables based on model defined in formula
y, X = model_matrix(flex, data)
n, p_flex = X.shape

model = make_pipeline(StandardScaler(), LinearRegression())
# Lots of learners work best with variables on a common scale. StandardScaler()
# standardizes the data. This can be very important with high order polynomials.

# Cross validation.
flexlm = cross_validate(model, X, y,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('Flex LM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-flexlm['test_neg_mean_squared_error'].mean()),
          m2=1-(-flexlm['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_flex))

print('Flex LM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-flexlm['test_neg_mean_absolute_error'].mean(),
          m2=1-(-flexlm['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_flex))

#####################################################################
# Model for log-price
logy = data['log_price']

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation. We're just going to look at MSE from here on out
flexloglm = cross_validate(model, X, logy,
                       scoring = scoring, cv=cvsplit)
print('Flex LogLM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-flexloglm['test_expmse'].mean()),
          m2=1-(-flexloglm['test_expmse'].mean()/benchMSE),
          m3=p_flex))

print('Flex LogLM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-flexloglm['test_expmae'].mean(),
          m2=1-(-flexloglm['test_expmae'].mean()/benchMAE),
          m3=p_flex))


**Aside:** I cheated in my definition of the scorer for the log models to prevent numeric issues in silly models (like this one). The actual performance is much worse.

We're not going to add these performance metrics to our table because they're awful and make the table unreadable.

It looks like the flexible model does a really bad job.

**Questions:**
1. I'm only showing validation sample performance measures. How do we think in-sample validation measures line up?
2. Does the very poor validation performance of the "interactions" model mean nothing in the model adds predictive power beyond the other models?
3. Are we sure we have tried everything we want? There are lots of "intermediate" models?
4. Should we have even more interactions or allow for higher order nonlinearity? How do we specify all the choices?

## Lasso

A popular approach to estimating linear models when there are many potential variables is the *Lasso*.

Recall that conventional linear model estimates are obtained by solving

$$\min \sum_{i} (y_i - b_0 - b_1 X_1 - ... - b_p X_p)^2$$

where the sum is over all observations in the training data. That is, we try to find the linear prediction rule for $Y$ that is as close as possible to $Y$ in the training data.

Lasso solves a similar problem

$$\min \sum_{i} (y_i - b_0 - b_1 X_1 - ... - b_p X_p)^2 + \lambda \sum_j |b_j|.$$

The key thing we have added is a *penalty term* $\lambda \sum_j |b_j|$. Intuitively, this penalty says that if you want to move a coefficient $b_j$ away from 0 to improve the in-sample fit, you also have to pay a cost introduced by the penalty term.

In practice, the presence of the penalty may lead to variables being excluded from the model. Intuitively, if the benefit of moving a $b_j$ away from 0 is smaller than the cost, the coefficient will be left at 0.

When we do Lasso, we have to choose the *tuning parameter* $\lambda$ which controls the cost of increasing the size of coefficients. We choose $\lambda$ by trying several values and choosing the best according to cross-validation (or just validation).

[Aside: There are MANY other penalized regression procedures. Another popular one is Ridge. There's also grouped Lasso, Elastic Net, ... The basic idea of all of them is to guard against overfitting by penalizing large values of coefficients. Not all of them do variable selection.]


Let's look at how Lasso does in our example. We have to start from a model that has the universe of variables that could matter. We'll start from both the polynomial and interactive models and see what happens.

# Lasso with baseline model

We're going to repeat the exercise but now use Lasso. Lasso requires us to choose how much we want it "to cost" to increase a parameter's magnitude (i.e. to move it away from zero). We're going to choose this by trying a bunch out and taking the best one. Ideally, we'd do this with another validation dataset.

In [ ]:
# Create variables from our baseline model (defined in formula base)
y, X = model_matrix(base, data)
n, p_base = X.shape

# Sequence of penalty parameters to try
ub = np.std(y)*np.sqrt(2*np.log(2*p_base/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/100, stop=ub, num=25)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation. Set up parameter we are trying to optimize.
parameters = {'lasso__alpha':lambdas.ravel()}

baselas = GridSearchCV(model, parameters,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'),
                       refit = 'neg_mean_squared_error', cv=cvsplit)
baselas.fit(X, y)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter
# Could also look at MAE minimizing, but didn't do it here.
lranks = baselas.cv_results_.get('rank_test_neg_mean_squared_error')
cvmse_baselas = -baselas.cv_results_.get('mean_test_neg_mean_squared_error')
cvmae_baselas = -baselas.cv_results_.get('mean_test_neg_mean_absolute_error')

bestmse_baselas = cvmse_baselas[lranks == 1].min()
bestmae_baselas = cvmae_baselas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = baselas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_baselas_use = sum(coefficients != 0)

# Print out summary of performance
print('Baseline Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_baselas),
          m2=1-(bestmse_baselas/benchMSE)))

print('Baseline Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_baselas,
          m2=1-(bestmae_baselas/benchMAE)))

print("p: ", p_base)
print("p_baselas_use: ", p_baselas_use)


In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Baseline Lasso',
                                       'RMSE': np.sqrt(bestmse_baselas),
                                       'R2 - MSE': 1-(bestmse_baselas/benchMSE),
                                       'MAE': bestmae_baselas,
                                       'R2 - MAE': 1-(bestmae_baselas/benchMAE),
                                       'p': p_base,
                                       'p_use': p_baselas_use}, ignore_index=True)

model_results

In [ ]:
# Which variables do we use
print(X.columns[lasso_model.coef_ != 0])

# Which do we drop
print(X.columns[lasso_model.coef_ == 0])


In [ ]:
# Plot the performance metrics across values of the tuning parameter
cvrmse_baselas = np.sqrt(cvmse_baselas)
lambdas = baselas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_baselas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_baselas, label='CV MAE')
plt.axvline(baselas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.legend()
plt.show()

Let's change and look at `log_price` as the dependent variable.

In [ ]:
# Log price model

# Sequence of penalty parameters to try
ub = np.std(logy)*np.sqrt(2*np.log(2*p_base/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/100, stop=ub, num=25)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

baseloglas = GridSearchCV(model, parameters,
                       scoring = scoring,
                       refit = 'expmse', cv=cvsplit)

baseloglas.fit(X, logy)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter. Note that we are looking at MSE for PRICE!, not
# log(price)
# Could also look at MAE minimizing, but didn't do it here.
lranks = baseloglas.cv_results_.get('rank_test_expmse')
cvlogmse_baseloglas = -baseloglas.cv_results_.get('mean_test_mse')
cvlogmae_baseloglas = -baseloglas.cv_results_.get('mean_test_mae')
cvmse_baseloglas = -baseloglas.cv_results_.get('mean_test_expmse')
cvmae_baseloglas = -baseloglas.cv_results_.get('mean_test_expmae')

bestmse_baseloglas = cvmse_baseloglas[lranks == 1].min()
bestmae_baseloglas = cvmae_baseloglas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = baseloglas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_baseloglas_use = sum(coefficients != 0)

# Print out summary of performance
print('Log Poly Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_baseloglas),
          m2=1-(bestmse_baseloglas/benchMSE)))

print('Log Poly Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_baseloglas,
          m2=1-(bestmae_baseloglas/benchMAE)))

print("p: ", p_base)
print("p_baseloglas_use: ", p_baseloglas_use)

In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Baseline logLasso',
                                       'RMSE': np.sqrt(bestmse_baseloglas),
                                       'R2 - MSE': 1-(bestmse_baseloglas/benchMSE),
                                       'MAE': bestmae_baseloglas,
                                       'R2 - MAE': 1-(bestmae_baseloglas/benchMAE),
                                       'p': p_base,
                                       'p_use': p_baseloglas_use}, ignore_index=True)

model_results

In [ ]:
# Plot the performance metrics across values of the tuning parameter
cvlogrmse_baseloglas = np.sqrt(cvlogmse_baseloglas)
lambdas = baseloglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvlogrmse_baseloglas, label='CV RMSE')
plt.semilogx(lambdas, cvlogmae_baseloglas, label='CV MAE')
plt.axvline(baseloglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for log(price)")
plt.legend()
plt.show()

In [ ]:
# Plot the performance metrics for PRICE across values of the tuning parameter
cvrmse_baseloglas = np.sqrt(cvmse_baseloglas)
lambdas = baseloglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_baseloglas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_baseloglas, label='CV MAE')
plt.axvline(baseloglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for price")
plt.legend()
plt.show()

In [ ]:
# Variables in the baseline log model estimated via lasso and estimated coefficients
baseloglas_coefs = pd.DataFrame(data={'Variable Names': X.columns[lasso_model.coef_ != 0], 'Coefficient': lasso_model.coef_[lasso_model.coef_ != 0]})
print(baseloglas_coefs.to_markdown())

# Lasso with flexible model

**(This block takes ~ 5 minutes to run.)**

In [ ]:
# Price prediction

# Create variables based on model defined in formula
y, X = model_matrix(flex, data)
n, p_flex = X.shape

# Sequence of penalty parameters to try
ub = np.std(y)*np.sqrt(2*np.log(2*p_flex/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/10, stop=ub, num=15)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

flexlas = GridSearchCV(model, parameters,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'),
                       refit = 'neg_mean_squared_error', cv=cvsplit)
flexlas.fit(X, y)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter
# Could also look at MAE minimizing, but didn't do it here.
lranks = flexlas.cv_results_.get('rank_test_neg_mean_squared_error')
cvmse_flexlas = -flexlas.cv_results_.get('mean_test_neg_mean_squared_error')
cvmae_flexlas = -flexlas.cv_results_.get('mean_test_neg_mean_absolute_error')

bestmse_flexlas = cvmse_flexlas[lranks == 1].min()
bestmae_flexlas = cvmae_flexlas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = flexlas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_flexlas_use = sum(coefficients != 0)

# Print out summary of performance
print('Level Interactions Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_flexlas),
          m2=1-(bestmse_flexlas/benchMSE)))

print('Level Interactions Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_flexlas,
          m2=1-(bestmae_flexlas/benchMAE)))

print("p_int: ", p_flex)
print("p_flexlas_use: ", p_flexlas_use)


In [ ]:
# Plot the performance metrics across values of the tuning parameter
cvrmse_flexlas = np.sqrt(cvmse_flexlas)
lambdas = flexlas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_flexlas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_flexlas, label='CV MAE')
plt.axvline(flexlas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Interactions Lasso',
                                       'RMSE': np.sqrt(bestmse_flexlas),
                                       'R2 - MSE': 1-(bestmse_flexlas/benchMSE),
                                       'MAE': bestmae_flexlas,
                                       'R2 - MAE': 1-(bestmae_flexlas/benchMAE),
                                       'p': p_flex,
                                       'p_use': p_flexlas_use}, ignore_index=True)

model_results

In [ ]:
# Variables in the flexible model estimated via lasso and estimated coefficients
flexlas_coefs = pd.DataFrame(data={'Variable Names': X.columns[lasso_model.coef_ != 0], 'Coefficient': lasso_model.coef_[lasso_model.coef_ != 0]})
print(flexlas_coefs.to_markdown())

**Questions:**

1. We have a list of variables and coefficients. Is the model particularly interpretable?
2. Should we conclude we have the "correct" variables?

Let's finish this section by looking at `log_price`

In [ ]:
# log price prediction

# Sequence of penalty parameters to try
ub = np.std(logy)*np.sqrt(2*np.log(2*p_flex/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/10, stop=ub, num=15)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

flexloglas = GridSearchCV(model, parameters,
                       scoring = scoring,
                       refit = 'expmse', cv=cvsplit)
flexloglas.fit(X, logy)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter. MSE based on predicting PRICE not LOGPRICE!
lranks = flexloglas.cv_results_.get('rank_test_expmse')
cvlogmse_flexloglas = -flexloglas.cv_results_.get('mean_test_mse')
cvlogmae_flexloglas = -flexloglas.cv_results_.get('mean_test_mae')
cvmse_flexloglas = -flexloglas.cv_results_.get('mean_test_expmse')
cvmae_flexloglas = -flexloglas.cv_results_.get('mean_test_expmae')

bestmse_flexloglas = cvmse_flexloglas[lranks == 1].min()
bestmae_flexloglas = cvmae_flexloglas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = flexloglas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_flexloglas_use = sum(coefficients != 0)

# Print out summary of performance
print('Log Poly Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_flexloglas),
          m2=1-(bestmse_flexloglas/benchMSE)))

print('Log Poly Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_flexloglas,
          m2=1-(bestmae_flexloglas/benchMAE)))

print("p: ", p_flex)
print("p_flexloglas_use: ", p_flexloglas_use)

In [ ]:
# Plot the performance metrics for PRICE across values of the tuning parameter
cvrmse_flexloglas = np.sqrt(cvmse_flexloglas)
lambdas = flexloglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_flexloglas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_flexloglas, label='CV MAE')
plt.axvline(flexloglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for price")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Interactions logLasso',
                                       'RMSE': np.sqrt(bestmse_flexloglas),
                                       'R2 - MSE': 1-(bestmse_flexloglas/benchMSE),
                                       'MAE': bestmae_flexloglas,
                                       'R2 - MAE': 1-(bestmae_flexloglas/benchMAE),
                                       'p': p_flex,
                                       'p_use': p_flexloglas_use}, ignore_index=True)

model_results

In [ ]:
# Variables in the flexible log model estimated via lasso and estimated coefficients
flexloglas_coefs = pd.DataFrame(data={'Variable Names': X.columns[lasso_model.coef_ != 0], 'Coefficient': lasso_model.coef_[lasso_model.coef_ != 0]})
print(flexloglas_coefs.to_markdown())

**Questions:**

1. We have a list of variables and coefficients. Is the model particularly interpretable?
2. Should we conclude we have the "correct" variables?
3. What if there are things we didn't try? How far should we go? Maybe we should have looked at higher order interactions (i.e. multiplying more than two variables together), other transformations, ...

**Miscellaneous comments:**

When we moved to Lasso, we could have included `sqft_above` and `sqft_basement` in the model along with `sqft_living` and let the data decide which are more useful. (For example, maybe basement square feet are not as valuable as above ground square feet, which can't be picked up when only `sqft_living` is included.)

We haven't considered `statezip` or `city` yet.

Penalized quantile regression (e.g. "lasso for LAV") exists and is available in `sklearn`'s `QuantileRegressor`.







**Final Comment on Linear Models**

Linear (and related) models are useful, but they do not do *feature learning*. I.e. they rely on the analyst to specify all the relevant features **and their transformations** before building the predictive model. In settings where there is strong intuition for what is important and how it relates to the target, this is not a huge problem. In setting with limited data, simple models (e.g. ignore transformations of continuous regressors and just include them as is) tend to work well.

Rather than require pre-specification of all input features, most ML methods leverage the power of machines and clever algorithms to simultaneously **learn feature representations** and build the model for making predictions.

The ability of ML algorithms to learn feature representations without needing prespecification is practically very important in complex environments or settings with large amounts of data. Even in our relatively simple house price example, we can already see that relying on prespecifying everything can be daunting.

Often (though not always) these models outperform methods that do not do feature learning in terms of predictive performance. Even if they do not "beat" more traditional methods, they almost always come with the benefit of alleviating the analyst's burden of choice.

# Regression Trees

Rather than try to build all the relevant features ourselves. Let's fit a regression *tree* instead.

The basic idea of a regression tree is to chop the data into rectangles based on the raw input variables and then fit a simple model - we'll just use the sample mean.

The way we'll chop things into rectangles is by making binary splits of the data. By making chops of the data based on binary decisions, we simultaneously uncover nonlinearity (including interactions) and which variables are most informative in making our predictions. We do not have to prespecify the types of nonlinearity we think might matter, we just learn it from the data.

Rather than try to talk about it, it's probably easier to just see how it works in our little example.



Let's start by pulling together a dataset that we will use for our trees. We'll focus on the same variables we used in our linear models. Note that we'll leave `sqft_above` and `sqft_basement` in as the tree can choose which representation of square footage it wants to use to best predict the outcome.

In [ ]:
data_tree = data.drop(columns=['city','statezip','renovated_flag','basement_flag'])

# Let's make our outcome variables and predictors
X = data_tree.drop(columns=['price','log_price'])
price = data_tree['price']
lprice = data_tree['log_price']

Because we're just illustrating, we'll start by making a tree with just one split using the full data set.

In [ ]:
# Let's fit a tree to price with 1 split = 2 leaves
tree1 = DecisionTreeRegressor(max_leaf_nodes = 2)
tree1.fit(X, price)
plot_tree(tree1, feature_names=X.columns)
plt.show()

The first split in our tree is at `sqft_living` = 2915. We predict houses less than or equal to 2915 square feet to have price \$457,977 and houses with more than 2915 square feet to have price \$993,690. (The predictions are just the sample mean price of houses smaller than 2915 square feet and sample mean of price of houses larger than 2915 square feet.)

This split was determined by looking at every possible split of every possible variable and asking which split provides the best prediction rule for `price` in the terms of having the minimum MSE.

What happens when we want to have a prediction rule with 3 terminal nodes?

In [ ]:
# Let's fit a tree with 2 splits = 3 leaves
tree2 = DecisionTreeRegressor(max_leaf_nodes = 3)
tree2.fit(X, y)
plot_tree(tree2, feature_names=X.columns)
plt.show()

We see that we have introduced another split along `sqft_living`. That we would split again on this variable was not predetermined. We tried every possible split along every possible variable of the subsample of houses with `sqft_living` $\leq$ 2915 and every possible split along every possible variable of the subsample of houses with `sqft_living` $>$ 2915. Of all these possible choices, the best split, in the sense of reducing MSE of predicting `price`, was to split again at `sqft_living` = 6140.

Our prediction rule now has three points, houses $\leq$ 2915 square feet are predicted to have price \$457,977. Houses with 2915 $<$ `sqft_living` $\leq$ 6140 are predicted to have price \$954,056. Houses greater than 6140 square feet are predicted to have price \$2,873,812.

We can, of course, keep going.


In [ ]:
# Make the figure bigger than default so it's easier to read
width = 10
height = 7
plt.figure(figsize=(width, height))

# Let's fit a tree with 3 splits = 4 leaves
tree3 = DecisionTreeRegressor(max_leaf_nodes = 4)
tree3.fit(X, y)
plot_tree(tree3, feature_names=X.columns)
plt.show()

Let's make one more plot to show how the regression tree is uncovering nonlinearity without having to prespecify transformations of the features.

In [ ]:
# Predict
yfit = tree3.predict(X)
xplot = np.sort(X['sqft_living'])
yplot = np.sort(yfit)

# Plot
plt.figure()
plt.plot(xplot, yplot, linewidth=2)
plt.xlabel("Square Footage")
plt.ylabel("Price")
plt.title("Decision Tree Regression with Four Leaves")
plt.show()

Basic tree models correspond to *step functions*. Step functions are simple, though not necessarily the best for lots of outcomes where we think *continuity* makes sense. We'll worry about this later.

Let's look at one more larger tree to see how we also pick up interactions.

In [ ]:
# Make the figure bigger than default so it's easier to read
width = 14
height = 10
plt.figure(figsize=(width, height))

# Let's fit a tree with 8 leaves
tree8 = DecisionTreeRegressor(max_leaf_nodes = 8)
tree8.fit(X, y)
plot_tree(tree8, feature_names=X.columns)
plt.show()

We can (and will) continue, though we'll stop showing the picture every time. The basic idea of trees is very simple. We keep make binary splits of the data where each split improves the fit to the training data. We can fit the training data very well with enough splits - though that's unlikely to generalize well.

We choose which tree we want to use by (cross-) validation. There are lots of ways we could specify and train trees. Rather than look at number of leaves, we could look at depth. We might want to specify a minimum leaf size to avoid leaves with very few observations.

We're just going to look at cross-validating over number of leaves and not worry about other details.

In [ ]:
# Parameter we want to choose based on cross-validation performance - number of leaves
parameters = {'max_leaf_nodes':range(2,101)}

scoring = {'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

tree = DecisionTreeRegressor()
base_tree = GridSearchCV(tree, parameters, scoring = scoring, cv = cvsplit, refit = 'mse')
base_tree.fit(X, price)

# Plot CV performance
leaves = base_tree.cv_results_.get('param_max_leaf_nodes')
leaves = leaves.tolist()

lranks = base_tree.cv_results_.get('rank_test_mse')
cvmse_tree = -base_tree.cv_results_.get('mean_test_mse')
cvmae_tree = -base_tree.cv_results_.get('mean_test_mae')

bestmse_base_tree = cvmse_tree[lranks == 1].min()
bestmae_base_tree = cvmae_tree[lranks == 1].min()

plt.plot(leaves, np.sqrt(cvmse_tree), label = 'RMSE')
plt.plot(leaves, cvmae_tree, label = 'MAE')
plt.axvline(base_tree.best_params_.get('max_leaf_nodes'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel("Number of leaves")
plt.ylabel("Cross-validation RMSE")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our results table
model_results = model_results._append({'Model': 'Decision Tree',
                                       'RMSE': np.sqrt(bestmse_base_tree),
                                       'R2 - MSE': 1-(bestmse_base_tree/benchMSE),
                                       'MAE': bestmae_base_tree,
                                       'R2 - MAE': 1-(bestmae_base_tree/benchMAE),
                                       'p': '-',
                                       'p_use': base_tree.best_params_.get('max_leaf_nodes')},
                                       ignore_index=True)

model_results

What does our estimated tree look like?

In [ ]:
width = 15
height = 15
plt.figure(figsize=(width, height))

# Let's plot the cv minimizing tree
plot_tree(base_tree.best_estimator_, feature_names=X.columns)
plt.show()

**We'd need a pretty big screen to see what's going on here.**

Looks like lots of results with similar performance and highly variable across number of leaves. Default behavior in the software allows for very small leaves. For example, we can see from the estimated tree that we have leaves with just one observation. We might want to add more stability be ruling out small leaves.

In [ ]:
# Parameter we want to choose based on cross-validation performance - number of leaves
parameters = {'max_leaf_nodes':range(2,101)}

scoring = {'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

# Let's make it so each leaf must have 10 or more observations
tree = DecisionTreeRegressor(min_samples_leaf=10)
tree10 = GridSearchCV(tree, parameters, scoring = scoring, cv = cvsplit, refit = 'mse')
tree10.fit(X, price)

leaves = tree10.cv_results_.get('param_max_leaf_nodes')
leaves = leaves.tolist()

lranks = tree10.cv_results_.get('rank_test_mse')
cvmse_tree = -tree10.cv_results_.get('mean_test_mse')
cvmae_tree = -tree10.cv_results_.get('mean_test_mae')

bestmse_tree10 = cvmse_tree[lranks == 1].min()
bestmae_tree10 = cvmae_tree[lranks == 1].min()

plt.plot(leaves, np.sqrt(cvmse_tree), label = 'RMSE')
plt.plot(leaves, cvmae_tree, label = 'MAE')
plt.axvline(tree10.best_params_.get('max_leaf_nodes'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel("Number of leaves")
plt.ylabel("Cross-validation RMSE")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our results table
model_results = model_results._append({'Model': 'Decision Tree 10',
                                       'RMSE': np.sqrt(bestmse_tree10),
                                       'R2 - MSE': 1-(bestmse_tree10/benchMSE),
                                       'MAE': bestmae_tree10,
                                       'R2 - MAE': 1-(bestmae_tree10/benchMAE),
                                       'p': '-',
                                       'p_use': tree10.best_params_.get('max_leaf_nodes')},
                                       ignore_index=True)

model_results

As we did before, we can also use `log_price` as dependent variable.

In [ ]:
# Parameter we want to choose based on cross-validation performance - number of leaves
parameters = {'max_leaf_nodes':range(2,151)}

scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

tree = DecisionTreeRegressor()
log_tree = GridSearchCV(tree, parameters, scoring = scoring, cv = cvsplit, refit = 'expmse')
log_tree.fit(X, lprice)

# Plot CV performance
leaves = log_tree.cv_results_.get('param_max_leaf_nodes')
leaves = leaves.tolist()

lranks = log_tree.cv_results_.get('rank_test_expmse')
cvmse_tree = -log_tree.cv_results_.get('mean_test_expmse')
cvmae_tree = -log_tree.cv_results_.get('mean_test_expmae')

bestmse_log_tree = cvmse_tree[lranks == 1].min()
bestmae_log_tree = cvmae_tree[lranks == 1].min()

plt.plot(leaves, np.sqrt(cvmse_tree), label = 'RMSE')
plt.plot(leaves, cvmae_tree, label = 'MAE')
plt.axvline(log_tree.best_params_.get('max_leaf_nodes'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel("Number of leaves")
plt.ylabel("Cross-validation RMSE")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our results table
model_results = model_results._append({'Model': 'Decision Tree - Log',
                                       'RMSE': np.sqrt(bestmse_log_tree),
                                       'R2 - MSE': 1-(bestmse_log_tree/benchMSE),
                                       'MAE': bestmae_log_tree,
                                       'R2 - MAE': 1-(bestmae_log_tree/benchMAE),
                                       'p': '-',
                                       'p_use': log_tree.best_params_.get('max_leaf_nodes')},
                                       ignore_index=True)

model_results

In [ ]:
# Parameter we want to choose based on cross-validation performance - number of leaves
parameters = {'max_leaf_nodes':range(2,151)}

scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

tree = DecisionTreeRegressor(min_samples_leaf=10)
log_tree10 = GridSearchCV(tree, parameters, scoring = scoring, cv = cvsplit, refit = 'expmse')
log_tree10.fit(X, lprice)

# Plot CV performance
leaves = log_tree10.cv_results_.get('param_max_leaf_nodes')
leaves = leaves.tolist()

lranks = log_tree10.cv_results_.get('rank_test_expmse')
cvmse_tree = -log_tree10.cv_results_.get('mean_test_expmse')
cvmae_tree = -log_tree10.cv_results_.get('mean_test_expmae')

bestmse_log_tree10 = cvmse_tree[lranks == 1].min()
bestmae_log_tree10 = cvmae_tree[lranks == 1].min()

plt.plot(leaves, np.sqrt(cvmse_tree), label = 'RMSE')
plt.plot(leaves, cvmae_tree, label = 'MAE')
plt.axvline(log_tree10.best_params_.get('max_leaf_nodes'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel("Number of leaves")
plt.ylabel("Cross-validation RMSE")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our results table
model_results = model_results._append({'Model': 'Decision Tree10 - Log',
                                       'RMSE': np.sqrt(bestmse_log_tree10),
                                       'R2 - MSE': 1-(bestmse_log_tree10/benchMSE),
                                       'MAE': bestmae_log_tree10,
                                       'R2 - MAE': 1-(bestmae_log_tree10/benchMAE),
                                       'p': '-',
                                       'p_use': log_tree10.best_params_.get('max_leaf_nodes')},
                                       ignore_index=True)

model_results

The tree models work a bit worse than the linear models here.

So, why do people like tree models?

1. They are simple and fast. They don't require much thought. Scale and monotone transformations of the features don't matter.
2. They underlie more complex procedures that we'll talk about next.
3. They don't require pre-specifying all the things we think might matter.

Finally, remember you should try out linear models as one of your prediction rules in many cases - certainly with small numbers of observations and standard numeric data. (Relatively simple) Linear models will often work well in that setting.

# Last thing - What about our categorical variables?



Unordered categorical variables with many categories relative to the number of observations (so there are not many observations within each category) are a challenge to many supervised learners and one of the spots where linear models do pretty well.

Let's try a couple of our models out with the categorical variables `city` and `statezip`.

# Baseline model including `city`

In [ ]:
# Specify model as formula
# Putting a variable inside "C" will make the variable be treated as categorical
# by turning it into dummy variables
# Putting a variable inside "poly" will a create a polynomial in that variable
# Putting v1:v2 creates the interaction between variables v1 and v2
basecity = ("price ~ poly(bathrooms, degree=2, raw=True) + poly(bedrooms, degree=2, raw=True)"
             " + poly(sqft_living, degree=2, raw=True) + poly(sqft_lot, degree=2, raw=True) + poly(floors, degree=2, raw=True) "
            "+ waterfront + poly(view, degree=2, raw=True) + poly(condition, degree=2, raw=True) + "
            "poly(yr_built, degree=2, raw=True) + poly(yr_renovated, degree=2, raw=True) + C(renovated_flag) + C(basement_flag)"
            " + sqft_living:(bedrooms+bathrooms) + C(city)")

# Create variables based on model defined in formula
y, X = model_matrix(basecity, data)
n, p_basecity = X.shape

# Cross validation.
basecitylm = cross_validate(LinearRegression(), X, y,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('City LM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-basecitylm['test_neg_mean_squared_error'].mean()),
          m2=1-(-basecitylm['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_basecity))

print('City LM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-basecitylm['test_neg_mean_absolute_error'].mean(),
          m2=1-(-basecitylm['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_basecity))

#####################################################################
# Model for log-price
logy = data['log_price']

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation.
basecityloglm = cross_validate(LinearRegression(), X, logy,
                       scoring = scoring, cv=cvsplit)
print('City logLM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-basecityloglm['test_expmse'].mean()),
          m2=1-(-basecityloglm['test_expmse'].mean()/benchMSE),
          m3=p_basecity))

print('City logLM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-basecityloglm['test_expmae'].mean(),
          m2=1-(-basecityloglm['test_expmae'].mean()/benchMAE),
          m3=p_basecity))


# Baseline model including `statezip`

In [ ]:
# Specify model as formula
# Putting a variable inside "C" will make the variable be treated as categorical
# by turning it into dummy variables
# Putting a variable inside "poly" will a create a polynomial in that variable
# Putting v1:v2 creates the interaction between variables v1 and v2
basezip = ("price ~ poly(bathrooms, degree=2, raw=True) + poly(bedrooms, degree=2, raw=True)"
             " + poly(sqft_living, degree=2, raw=True) + poly(sqft_lot, degree=2, raw=True) + poly(floors, degree=2, raw=True) "
            "+ waterfront + poly(view, degree=2, raw=True) + poly(condition, degree=2, raw=True) + "
            "poly(yr_built, degree=2, raw=True) + poly(yr_renovated, degree=2, raw=True) + C(renovated_flag) + C(basement_flag)"
            " + sqft_living:(bedrooms+bathrooms) + C(statezip)")

# Create variables based on model defined in formula
y, X = model_matrix(basezip, data)
n, p_basezip = X.shape

# Cross validation.
baseziplm = cross_validate(LinearRegression(), X, y,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('City LM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baseziplm['test_neg_mean_squared_error'].mean()),
          m2=1-(-baseziplm['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_basezip))

print('City LM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baseziplm['test_neg_mean_absolute_error'].mean(),
          m2=1-(-baseziplm['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_basezip))

#####################################################################
# Model for log-price
logy = data['log_price']

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation.
baseziploglm = cross_validate(LinearRegression(), X, logy,
                       scoring = scoring, cv=cvsplit)
print('City logLM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-baseziploglm['test_expmse'].mean()),
          m2=1-(-baseziploglm['test_expmse'].mean()/benchMSE),
          m3=p_basezip))

print('City logLM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-baseziploglm['test_expmae'].mean(),
          m2=1-(-baseziploglm['test_expmae'].mean()/benchMAE),
          m3=p_basezip))


# Baseline model including full interaction of `city` and `statezip`

**This cell takes ~ 10 minutes**

In [ ]:
# Specify model as formula
# Putting a variable inside "C" will make the variable be treated as categorical
# by turning it into dummy variables
# Putting a variable inside "poly" will a create a polynomial in that variable
# Putting v1:v2 creates the interaction between variables v1 and v2
baseboth = ("price ~ poly(bathrooms, degree=2, raw=True) + poly(bedrooms, degree=2, raw=True)"
             " + poly(sqft_living, degree=2, raw=True) + poly(sqft_lot, degree=2, raw=True) + poly(floors, degree=2, raw=True) "
            "+ waterfront + poly(view, degree=2, raw=True) + poly(condition, degree=2, raw=True) + "
            "poly(yr_built, degree=2, raw=True) + poly(yr_renovated, degree=2, raw=True) + C(renovated_flag) + C(basement_flag)"
            " + sqft_living:(bedrooms+bathrooms) + C(city)*C(statezip)")

# Create variables based on model defined in formula
y, X = model_matrix(baseboth, data)
n, p_baseboth = X.shape

# Cross validation.
basebothlm = cross_validate(LinearRegression(), X, y,
                       scoring = ('neg_mean_squared_error','neg_mean_absolute_error'), cv=cvsplit)
print('City LM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-basebothlm['test_neg_mean_squared_error'].mean()),
          m2=1-(-basebothlm['test_neg_mean_squared_error'].mean()/benchMSE),
          m3=p_baseboth))

print('City LM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-basebothlm['test_neg_mean_absolute_error'].mean(),
          m2=1-(-basebothlm['test_neg_mean_absolute_error'].mean()/benchMAE),
          m3=p_baseboth))

#####################################################################
# Model for log-price
logy = data['log_price']

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score}

# Cross validation.
basebothloglm = cross_validate(LinearRegression(), X, logy,
                       scoring = scoring, cv=cvsplit)
print('City logLM: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=np.sqrt(-basebothloglm['test_expmse'].mean()),
          m2=1-(-basebothloglm['test_expmse'].mean()/benchMSE),
          m3=p_baseboth))

print('City logLM: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}; p: {m3:=.0f}'
  .format(m1=-basebothloglm['test_expmae'].mean(),
          m2=1-(-basebothloglm['test_expmae'].mean()/benchMAE),
          m3=p_baseboth))


In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'City LM',
                                       'RMSE': np.sqrt(-basecitylm['test_neg_mean_squared_error'].mean()),
                                       'R2 - MSE': 1-(-basecitylm['test_neg_mean_squared_error'].mean()/benchMSE),
                                       'MAE': -basecitylm['test_neg_mean_absolute_error'].mean(),
                                       'R2 - MAE': 1-(-basecitylm['test_neg_mean_absolute_error'].mean()/benchMAE),
                                       'p': p_basecity,
                                       'p_use': p_basecity}, ignore_index=True)

model_results = model_results._append({'Model': 'City logLM',
                                       'RMSE': np.sqrt(-basecityloglm['test_expmse'].mean()),
                                       'R2 - MSE': 1-(-basecityloglm['test_expmse'].mean()/benchMSE),
                                       'MAE': -basecityloglm['test_expmae'].mean(),
                                       'R2 - MAE': 1-(-basecityloglm['test_expmae'].mean()/benchMAE),
                                       'p': p_basecity,
                                       'p_use': p_basecity}, ignore_index=True)

model_results = model_results._append({'Model': 'Zip LM',
                                       'RMSE': np.sqrt(-baseziplm['test_neg_mean_squared_error'].mean()),
                                       'R2 - MSE': 1-(-baseziplm['test_neg_mean_squared_error'].mean()/benchMSE),
                                       'MAE': -baseziplm['test_neg_mean_absolute_error'].mean(),
                                       'R2 - MAE': 1-(-baseziplm['test_neg_mean_absolute_error'].mean()/benchMAE),
                                       'p': p_basezip,
                                       'p_use': p_basezip}, ignore_index=True)

model_results = model_results._append({'Model': 'Zip logLM',
                                       'RMSE': np.sqrt(-baseziploglm['test_expmse'].mean()),
                                       'R2 - MSE': 1-(-baseziploglm['test_expmse'].mean()/benchMSE),
                                       'MAE': -baseziploglm['test_expmae'].mean(),
                                       'R2 - MAE': 1-(-baseziploglm['test_expmae'].mean()/benchMAE),
                                       'p': p_basezip,
                                       'p_use': p_basezip}, ignore_index=True)

model_results = model_results._append({'Model': 'Both LM',
                                       'RMSE': np.sqrt(-basebothlm['test_neg_mean_squared_error'].mean()),
                                       'R2 - MSE': 1-(-basebothlm['test_neg_mean_squared_error'].mean()/benchMSE),
                                       'MAE': -basebothlm['test_neg_mean_absolute_error'].mean(),
                                       'R2 - MAE': 1-(-basebothlm['test_neg_mean_absolute_error'].mean()/benchMAE),
                                       'p': p_baseboth,
                                       'p_use': p_baseboth}, ignore_index=True)

model_results = model_results._append({'Model': 'Both logLM',
                                       'RMSE': np.sqrt(-basebothloglm['test_expmse'].mean()),
                                       'R2 - MSE': 1-(-basebothloglm['test_expmse'].mean()/benchMSE),
                                       'MAE': -basebothloglm['test_expmae'].mean(),
                                       'R2 - MAE': 1-(-basebothloglm['test_expmae'].mean()/benchMAE),
                                       'p': p_baseboth,
                                       'p_use': p_baseboth}, ignore_index=True)


model_results


The categorical variables, particulary `statezip`, seems to be adding a lot of predictive power. **Remember: What information are these categorical variables likely capturing?**


# Lasso and trees with categorical variables

Rather than go through the full collection of models again. Let's just look at how adding these categorical variables affects a couple of the Lasso results and the trees.

## Lasso with baseline variables and statezip dummies

In [ ]:
# Lasso in the model with statezip, just going to look at predicing log(price)

# Create variables from our basezip model
y, X = model_matrix(basezip, data)
n, p_basezip = X.shape

# Model for log-price
logy = data['log_price']

# Sequence of penalty parameters to try
ub = np.std(logy)*np.sqrt(2*np.log(2*p_basezip/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/100, stop=ub, num=25)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

baseziploglas = GridSearchCV(model, parameters,
                       scoring = scoring,
                       refit = 'expmse', cv=cvsplit)

baseziploglas.fit(X, logy)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter. Note that we are looking at MSE for PRICE!, not
# log(price)
# Could also look at MAE minimizing, but didn't do it here.
lranks = baseziploglas.cv_results_.get('rank_test_expmse')
cvlogmse_baseziploglas = -baseziploglas.cv_results_.get('mean_test_mse')
cvlogmae_baseziploglas = -baseziploglas.cv_results_.get('mean_test_mae')
cvmse_baseziploglas = -baseziploglas.cv_results_.get('mean_test_expmse')
cvmae_baseziploglas = -baseziploglas.cv_results_.get('mean_test_expmae')

bestmse_baseziploglas = cvmse_baseziploglas[lranks == 1].min()
bestmae_baseziploglas = cvmae_baseziploglas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = baseziploglas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_baseziploglas_use = sum(coefficients != 0)

# Print out summary of performance
print('Zip Log Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_baseziploglas),
          m2=1-(bestmse_baseziploglas/benchMSE)))

print('Zip Log Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_baseziploglas,
          m2=1-(bestmae_baseziploglas/benchMAE)))

print("p: ", p_basezip)
print("p_use: ", p_baseziploglas_use)

In [ ]:
# Plot the performance metrics for PRICE across values of the tuning parameter
cvrmse_baseziploglas = np.sqrt(cvmse_baseziploglas)
lambdas = baseziploglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_baseziploglas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_baseziploglas, label='CV MAE')
plt.axvline(baseziploglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for price")
plt.legend()
plt.show()

## Lasso with baseline variables and statezip x city dummies

**This block takes ~10 minutes to run**

In [ ]:
# Lasso in the model with statezip X city, just going to look at predicing log(price)

# Create variables from our baseboth model
y, X = model_matrix(baseboth, data)
n, p_baseboth = X.shape

# Model for log-price
logy = data['log_price']

# Sequence of penalty parameters to try
ub = np.std(logy)*np.sqrt(2*np.log(2*p_baseboth/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/100, stop=ub, num=25)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

basebothloglas = GridSearchCV(model, parameters,
                       scoring = scoring,
                       refit = 'expmse', cv=cvsplit)

basebothloglas.fit(X, logy)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter. Note that we are looking at MSE for PRICE!, not
# log(price)
# Could also look at MAE minimizing, but didn't do it here.
lranks = basebothloglas.cv_results_.get('rank_test_expmse')
cvlogmse_basebothloglas = -basebothloglas.cv_results_.get('mean_test_mse')
cvlogmae_basebothloglas = -basebothloglas.cv_results_.get('mean_test_mae')
cvmse_basebothloglas = -basebothloglas.cv_results_.get('mean_test_expmse')
cvmae_basebothloglas = -basebothloglas.cv_results_.get('mean_test_expmae')

bestmse_basebothloglas = cvmse_basebothloglas[lranks == 1].min()
bestmae_basebothloglas = cvmae_basebothloglas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = basebothloglas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_basebothloglas_use = sum(coefficients != 0)

# Print out summary of performance
print('Both Log Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_basebothloglas),
          m2=1-(bestmse_basebothloglas/benchMSE)))

print('Both Log Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_basebothloglas,
          m2=1-(bestmae_basebothloglas/benchMAE)))

print("p: ", p_baseboth)
print("p_use: ", p_basebothloglas_use)

In [ ]:
# Plot the performance metrics for PRICE across values of the tuning parameter
cvrmse_basebothloglas = np.sqrt(cvmse_basebothloglas)
lambdas = basebothloglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_basebothloglas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_basebothloglas, label='CV MAE')
plt.axvline(basebothloglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for price")
plt.legend()
plt.show()

## Lasso with flexible variables and statezip x city dummies

For giggles, let's look at what happens when we take the "flexible" model and throw in the `statezip` x `city` dummies.

**This block takes ~10 minutes to run**

In [ ]:
# Lasso in the flexible model also adding statezip X city
# Just going to look at predicing log(price)
flex2 = ("log_price ~ poly(bathrooms, degree=4, raw=True) + poly(bedrooms, degree=4, raw=True)"
             " + poly(sqft_living, degree=6, raw=True) + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag)"
             " + poly(bathrooms, degree=4, raw=True):(poly(bedrooms, degree=4, raw=True)"
             " + poly(sqft_living, degree=6, raw=True) + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(bedrooms, degree=4, raw=True):(poly(sqft_living, degree=6, raw=True)"
             " + poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(sqft_living, degree=6, raw=True):(poly(sqft_lot, degree=4, raw=True) + poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(sqft_lot, degree=4, raw=True):(poly(floors, degree=4, raw=True)"
             " + waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(floors, degree=4, raw=True):(waterfront + C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + waterfront:(C(view) + C(condition)"
             " + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + C(view):(C(condition) + poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + C(condition):(poly(yr_built, degree=4, raw=True) + poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(yr_built, degree=4, raw=True):(poly(yr_renovated, degree=4, raw=True) + C(renovated_flag) + C(basement_flag))"
             " + poly(yr_renovated, degree=4, raw=True):(C(renovated_flag) + C(basement_flag))"
             " + C(renovated_flag):C(basement_flag) + C(city)*C(statezip)")


# Create variables based on model defined in formula
logy, X = model_matrix(flex2, data)
n, p_flex2 = X.shape

# Sequence of penalty parameters to try
ub = np.std(logy)*np.sqrt(2*np.log(2*p_flex2/.05))/np.sqrt(n)
lambdas = np.geomspace(start=ub/10, stop=ub*10, num=25)

# Let's do cross-validation in the training data to choose a value for lambda
model = make_pipeline(StandardScaler(), Lasso())
# "StandardScaler" is important here. It standardizes the data. Because Lasso
# depends on the coefficients of the linear model and the coefficients depend
# on the scale of the X's, you can get very different answers depending on how
# you choose to scale the variables.

# Cross validation
parameters = {'lasso__alpha':lambdas.ravel()}

# Set things up for custom scoring when using log(price)
scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

flex2loglas = GridSearchCV(model, parameters,
                       scoring = scoring,
                       refit = 'expmse', cv=cvsplit)

flex2loglas.fit(X, logy)

# Extract the performance measures and find the values at the MSE minimizing
# value of the tuning parameter. Note that we are looking at MSE for PRICE!, not
# log(price)
# Could also look at MAE minimizing, but didn't do it here.
lranks = flex2loglas.cv_results_.get('rank_test_expmse')
cvlogmse_flex2loglas = -flex2loglas.cv_results_.get('mean_test_mse')
cvlogmae_flex2loglas = -flex2loglas.cv_results_.get('mean_test_mae')
cvmse_flex2loglas = -flex2loglas.cv_results_.get('mean_test_expmse')
cvmae_flex2loglas = -flex2loglas.cv_results_.get('mean_test_expmae')

bestmse_flex2loglas = cvmse_flex2loglas[lranks == 1].min()
bestmae_flex2loglas = cvmae_flex2loglas[lranks == 1].min()

# Let's see how many variables are lasso model is using
# Get the best model
best_model = flex2loglas.best_estimator_

# Access Lasso model from the pipeline
lasso_model = best_model.named_steps['lasso']

# Retrieve coefficients
coefficients = lasso_model.coef_
p_flex2loglas_use = sum(coefficients != 0)

# Print out summary of performance
print('Both Log Lasso: CV RMSE: {m1:=.2f}; CV R2: {m2:=.3f}'
  .format(m1=np.sqrt(bestmse_flex2loglas),
          m2=1-(bestmse_flex2loglas/benchMSE)))

print('Both Log Lasso: CV MAE: {m1:=.2f}; CV MAE R2: {m2:=.3f}'
  .format(m1=bestmae_flex2loglas,
          m2=1-(bestmae_flex2loglas/benchMAE)))

print("p: ", p_flex2)
print("p_use: ", p_flex2loglas_use)

# Plot the performance metrics for PRICE across values of the tuning parameter
cvrmse_flex2loglas = np.sqrt(cvmse_flex2loglas)
lambdas = flex2loglas.cv_results_.get('param_lasso__alpha').tolist()

# Make plot
plt.semilogx(lambdas, cvrmse_flex2loglas, label='CV RMSE')
plt.semilogx(lambdas, cvmae_flex2loglas, label='CV MAE')
plt.axvline(flex2loglas.best_params_.get('lasso__alpha'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error Estimate")
plt.title("RMSE and MAE for price")
plt.legend()
plt.show()

In [ ]:
# Let's add these results to our result table
model_results = model_results._append({'Model': 'Zip logLasso',
                                       'RMSE': np.sqrt(bestmse_baseziploglas),
                                       'R2 - MSE': 1-(bestmse_baseziploglas/benchMSE),
                                       'MAE': bestmae_baseziploglas,
                                       'R2 - MAE': 1-(bestmae_baseziploglas/benchMAE),
                                       'p': p_basezip,
                                       'p_use': p_baseziploglas_use}, ignore_index=True)

model_results = model_results._append({'Model': 'City X Zip logLasso',
                                       'RMSE': np.sqrt(bestmse_basebothloglas),
                                       'R2 - MSE': 1-(bestmse_basebothloglas/benchMSE),
                                       'MAE': bestmae_basebothloglas,
                                       'R2 - MAE': 1-(bestmae_basebothloglas/benchMAE),
                                       'p': p_baseboth,
                                       'p_use': p_basebothloglas_use}, ignore_index=True)

model_results = model_results._append({'Model': 'Int + City X Zip logLasso',
                                       'RMSE': np.sqrt(bestmse_flex2loglas),
                                       'R2 - MSE': 1-(bestmse_flex2loglas/benchMSE),
                                       'MAE': bestmae_flex2loglas,
                                       'R2 - MAE': 1-(bestmae_flex2loglas/benchMAE),
                                       'p': p_flex2,
                                       'p_use': p_flex2loglas_use}, ignore_index=True)

model_results

# Tree with `statezip` and `city`

For use with trees, we are going to need to create dummy variables for `statezip` and `city`.

In [ ]:
data_tree = data.drop(columns=['renovated_flag','basement_flag'])
data_tree = pd.get_dummies(data_tree, columns=['statezip','city'])

# Let's make our outcome variables and predictors
X = data_tree.drop(columns=['price','log_price'])
price = data_tree['price']
lprice = data_tree['log_price']

Let's just look at a baseline tree for `log_price`.

In [ ]:
# Parameter we want to choose based on cross-validation performance - number of leaves
parameters = {'max_leaf_nodes':range(2,151)}

scoring = {'expmse': expmse_score,
           'expmae': expmae_score,
           'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error'}

tree = DecisionTreeRegressor()
log_bothtree = GridSearchCV(tree, parameters, scoring = scoring, cv = cvsplit, refit = 'expmse')
log_bothtree.fit(X, lprice)

# Plot CV performance
leaves = log_bothtree.cv_results_.get('param_max_leaf_nodes')
leaves = leaves.tolist()

lranks = log_bothtree.cv_results_.get('rank_test_expmse')
cvmse_bothtree = -log_bothtree.cv_results_.get('mean_test_expmse')
cvmae_bothtree = -log_bothtree.cv_results_.get('mean_test_expmae')

bestmse_log_bothtree = cvmse_bothtree[lranks == 1].min()
bestmae_log_bothtree = cvmae_bothtree[lranks == 1].min()

plt.plot(leaves, np.sqrt(cvmse_bothtree), label = 'RMSE')
plt.plot(leaves, cvmae_bothtree, label = 'MAE')
plt.axvline(log_bothtree.best_params_.get('max_leaf_nodes'),
            linestyle="--", color="black", label="CV estimate")
plt.xlabel("Number of leaves")
plt.ylabel("Cross-validation RMSE")
plt.legend()
plt.show()

# Let's add these results to our results table
model_results = model_results._append({'Model': 'Tree, zip X city - Log',
                                       'RMSE': np.sqrt(bestmse_log_bothtree),
                                       'R2 - MSE': 1-(bestmse_log_bothtree/benchMSE),
                                       'MAE': bestmae_log_bothtree,
                                       'R2 - MAE': 1-(bestmae_log_bothtree/benchMAE),
                                       'p': '-',
                                       'p_use': log_bothtree.best_params_.get('max_leaf_nodes')},
                                       ignore_index=True)

model_results

